In [ ]:
!pip install ta
!pip install yfinance

In [69]:
import yfinance as yf
from datetime import date
import pandas as pd
from ta.trend import SMAIndicator

In [70]:
def load_data(filepath):
    tickers = set()

    # load the data of all tickers in SGX to pass into yfinance
    file = open(filepath, "r")
    for line in file:
        line = line.strip()
        tickers.add(line)
    
    return tickers

def get_dividends(ticker):
    # Get pandas series of dividends
    info = yf.Ticker(ticker)
    return info.dividends

def get_price_history(ticker, period):
    # Get pandas df of prices to match to dividend payout date
    return yf.Ticker(ticker).history(period=period, auto_adjust=False)

def process_data(read_filepath, write_filepath):
    # load the data of all tickers in SGX to pass into yfinance
    read_file = open(read_filepath, "r")
    write_file = open(write_filepath, "a")
    for line in read_file:
        write_file.write(line[:4].strip() + ".SI\n")
    
    return

In [126]:
# returns a list of expected ex date and amount
def get_expected_ex_date(dividends):
    # Get today's date
    today = pd.Timestamp(date.today(), tz="UTC")

    # Get the ex-date of the stock based on last year's data
    previous_year_date = today.replace(year=today.year - 1)

    # if no dividends in the past 1-year period, unable to find expected ex date since they havent been giving out dividends for a year
    if dividends.empty:
# print("Dividends is empty.")
        return []
# else:
# print("Dividends is not empty!")
# print(dividends)

    # filter past dividend dates within a 1-year period and get first date (upcoming)
    filtered_dividends = dividends[dividends.index > previous_year_date]
# print("Filtered Dividends: ")
# print(filtered_dividends)

    # if they haven't paid dividends in the past year
    if filtered_dividends.empty:
# print("Filtered dividends is empty")
        return []

    expected_ex_date = filtered_dividends.index[0].date()
    expected_payout = filtered_dividends.iloc[0]
    
    result = []

    try:
        result.append(expected_ex_date.replace(year=expected_ex_date.year + 1))
        result.append(float(expected_payout))
    except:
        print("Error in changing expected ex date")
        return []

    return result

In [127]:
def get_trend(ticker_name, sma_short_window=10, sma_long_window=20, momentum_days=10, momentum_threshold=0.02):
    """
    Determines whether the stock is bullish, bearish, or in consolidation based on short-term trends
        - Takes into account momentum and whether SMA short or SMA long is higher
    
    returns a string: "Bullish", "Bearish", or "Consolidation". ("Unknown" if error)
    """
    # 1 month might not have 20 trading days, so we do 2 months time frame
    price_data = get_price_history(ticker_name, "2mo")
    
    if price_data.empty or 'Close' not in price_data:
        return "Unknown"

    # have to squeeze as its a 2d array
    close_prices = price_data['Close'].squeeze()

    # get sma of short window and long windows
    sma_short = SMAIndicator(close=close_prices, window=sma_short_window)
    sma_long = SMAIndicator(close=close_prices, window=sma_long_window)
    price_data['SMA_short'] = sma_short.sma_indicator()
    price_data['SMA_long'] = sma_long.sma_indicator()

    # get momentum (percentage change)
    price_data['Momentum'] = close_prices.pct_change(periods=momentum_days)

    # get latest value
    latest_sma_short = price_data['SMA_short'].iloc[-1]
    latest_sma_long = price_data['SMA_long'].iloc[-1]
    latest_momentum = price_data['Momentum'].iloc[-1]

    # bullish if sma short higher than sma long and momentum validates it, bearish if opposite
    if latest_sma_short > latest_sma_long and latest_momentum > momentum_threshold:
        return "Bullish"
    elif latest_sma_short < latest_sma_long and latest_momentum < -momentum_threshold:
        return "Bearish"
    else:
        return "Consolidation"

In [138]:
# FRONT PAGE
def get_stocks_dict_list(tickers):
    result = []
    id = 1

    # for each ticker in SGX, get relevant information into a dictionary and append to list
    for ticker_name in tickers:
        try:
            dividends = get_dividends(ticker_name)
            expected_dividend_info = get_expected_ex_date(dividends) # might return empty list if no dividends given
            
            # if empty, don't need process this ticker, it doesn't give dividends -- the method returns [] if dividends dont exist
            if not expected_dividend_info:
                continue

            ticker_info = yf.Ticker(ticker_name).info

            current_stock = {}
            current_stock["id"] = id
            id += 1
            current_stock["symbol"] = ticker_name
            
            # ticker might not have a variable longName in the api
            try: 
                current_stock["name"] = ticker_info['longName']
            except:
                continue

            current_stock["exDate"] = expected_dividend_info[0]
            current_stock["payout"] = expected_dividend_info[1]
            current_stock["trend"] = get_trend(ticker_name) # bearish, bullish or consolidation

            # dict["captured_yield"] = get_yield(ticker_name) # dividend capture strategy yield based on past data -- can remove, put in backtest page
            print(current_stock)
            result.append(current_stock)
        except:
            return result

    return result

In [139]:
filepath = "data/tickers.txt"
file = load_data(filepath)

In [ ]:
dict_list_of_stocks = get_stocks_dict_list(file)
print(dict_list_of_stocks)

In [142]:
import datetime
def default_serializer(obj):
    if isinstance(obj, (datetime.date, datetime.datetime)):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")


In [143]:
import json
output_filepath = "data/stock_list_data.json"
with open(output_filepath, "w", encoding="utf-8") as f:
    json.dump(dict_list_of_stocks, f, ensure_ascii=False, indent=2, default=default_serializer)